<a href="https://colab.research.google.com/github/adithyathecoder/underwater_image_enhancement/blob/main/Copy_of_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%%capture
!pip install timm

In [ ]:
import torch as T
from torch.nn import functional as F
from torch import nn
import cv2
from PIL import Image, ImageOps, ImageEnhance, __version__ as PILLOW_VERSION
import matplotlib.pyplot as plt
import timm
import torchvision
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler, SequentialSampler
from torchvision import transforms
import torchvision.transforms.functional as tvf
import numpy as np
import os
import time
import math
from prettytable import PrettyTable
import pandas as pd
from ast import literal_eval
from tqdm import tqdm
from matplotlib import pyplot as plt

device = "cuda"

In [ ]:
class Encoder(nn.Module):
    def __init__(self, backbone = 'resnet18', device = 'cuda'):
        super(Encoder, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained = True)
        self.List = list(self.backbone.children())[:-2]
        self.device = device
    def forward(self,X):
        X = X.to(self.device).float()
        outputs = []
        for i,layer in enumerate(self.List):
            X = layer(X)
            if i > 1:
                outputs.append(X)
        return outputs

class ResidualLayer(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels):
        super(ResidualLayer, self).__init__()
        self.resblock = nn.Sequential(nn.Conv2d(in_channels, out_channels,
                                                kernel_size=3, padding=1, bias=False),
                                      nn.ReLU(True),
                                      nn.Conv2d(out_channels, out_channels,
                                                kernel_size=1, bias=False))
        self.conv = nn.Conv2d(in_channels*2, out_channels,
                                                kernel_size=3, padding=1, bias=False)

    def forward(self, input_1,input_2):
        Y = input_1 + self.resblock(input_1)
        Y = T.cat((Y,input_2),dim=1)
        return self.conv(Y)

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.upsample = nn.Upsample(scale_factor=2, mode = 'bilinear')
        self.res1 = ResidualLayer(256,256)
        self.res2 = ResidualLayer(128,128)
        self.res3 = ResidualLayer(64,64)
        self.res4 = ResidualLayer(64,64)
        self.conv1 = nn.Conv2d(512,256,(3,3),padding = 1)  
        self.conv2 = nn.Conv2d(256,128,(3,3),padding = 1)  
        self.conv3 = nn.Conv2d(128,64,(3,3),padding = 1)
        self.conv4 = nn.Conv2d(64,64,(3,3),padding = 1)
        self.out = nn.Conv2d(64,3,(1,1))
        
    def forward(self,outputs):
        X = self.upsample(outputs[-1])
        X = F.relu(self.conv1(X))
        X = self.res1(X,outputs[-2])
        X = self.upsample(X)
        X = F.relu(self.conv2(X))
        X = self.res2(X,outputs[-3])
        X = self.upsample(X)
        X = F.relu(self.conv3(X))
        X = self.res3(X,outputs[-5])
        X = self.upsample(X)
        X = F.relu(self.conv4(X))
        X = self.res4(X,outputs[-6])
        return self.out(X)
    
class AutoEncoder(nn.Module):
    def __init__(self,device='cuda'):
        super(AutoEncoder,self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        self.to(device)
        
    def forward(self,X):
        X = self.encoder(X)
        X = self.decoder(X)
        return X

In [ ]:
train_dataset='/content/drive/MyDrive/Induction_training_files/raw'
test_dataset='/content/drive/MyDrive/Induction_training_files/test'
enhanced_dataset='/content/drive/MyDrive/Induction_training_files/enhanced'

In [ ]:
import os
a=os.listdir('/content/drive/MyDrive/Induction_training_files/raw')
print(a)
file_paths = []

import os

def get_filepaths(directory):
    
    file_paths = []  

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  

    return file_paths  


['704_img_.png', '101_img_.png', '102_img_.png', '104_img_.png', '106_img_.png', '105_img_.png', '108_img_.png', '107_img_.png', '113_img_.png', '118_img_.png', '117_img_.png', '116_img_.png', '120_img_.png', '121_img_.png', '122_img_.png', '124_img_.png', '125_img_.png', '123_img_.png', '133_img_.png', '131_img_.png', '130_img_.png', '135_img_.png', '136_img_.png', '137_img_.png', '138_img_.png', '141_img_.png', '145_img_.png', '144_img_.png', '149_img_.png', '148_img_.png', '146_img_.png', '151_img_.png', '150_img_.png', '152_img_.png', '157_img_.png', '154_img_.png', '155_img_.png', '153_img_.png', '156_img_.png', '158_img_.png', '159_img_.png', '161_img_.png', '162_img_.png', '163_img_.png', '164_img_.png', '165_img_.png', '166_img_.png', '169_img_.png', '171_img_.png', '170_img_.png', '174_img_.png', '172_img_.png', '175_img_.png', '177_img_.png', '176_img_.png', '178_img_.png', '180_img_.png', '179_img_.png', '183_img_.png', '185_img_.png', '187_img_.png', '188_img_.png', '189_im

In [ ]:
 path_test= get_filepaths("/content/drive/MyDrive/Induction_training_files/raw")
 import pandas as pd
 import tensorflow as tf
 from tensorflow import keras
 from __future__ import absolute_import, division, print_function
 df= pd.DataFrame({"raw":path_test}) 
 df.to_csv("small .csv")
 path_enhance = get_filepaths("/content/drive/MyDrive/Induction_training_files/enhanced")
 dp = pd.DataFrame({"raw":path_enhance})
 dp.to_csv("large .csv")
 #indata = pd.read_csv('small .csv')
 #zipped = pd.DataFrame({"enhance":path_enhance})
 #axis=1 indicates to concat the frames column wise
 #outdata = pd.concat([indata, zipped], axis=1)
 #we dont want headers and dont want the row labels
 #outdata.to_csv('small .csv',index=False)
checkpoint_path = '/content/model2.pth'
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path)

In [ ]:
class Data(Dataset):
    def __init__(self, dir_=None, input_size=(224,224), output_size=(112, 112),prob = 0.15):
        super().__init__()
        self.directory = dir_
        self.dataset = pd.read_csv(self.directory)
        self.input_size = input_size
        self.output_size = output_size
        self.prob = prob
        self.input_size_x = self.input_size[0]
        self.input_size_y = self.input_size[1]
        self.MODEL_SCALE = self.input_size[0]//self.output_size[0]
        self.preprocess = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __len__(self): return len(self.dataset)

    def __getitem_internal__(self, idx, preprocess=True):
        target = self.dataset.iloc[idx]
        noise_image= cv2.imread(target["raw"])
        noise_image = cv2.resize(noise_image,self.input_size)
        noise_image = transforms.ToTensor()(np.array(noise_image))
        dirname, basename = os.path.split(target["raw"])
        main,raws=os.path.split(dirname)
        path = os.path.join(main, 'enhanced', basename)
        rgb_image = cv2.imread(path) 
        rgb_image = cv2.resize(rgb_image,self.output_size)
        rgb_image = transforms.ToTensor()(np.array(rgb_image))
        return (noise_image,rgb_image)

    def __getitem__(self, idx):
        return self.__getitem_internal__(idx, True)
    
    def raw(self, idx):
        return self.__getitem_internal__(idx, False)
    

In [ ]:
#import pandas as pd
#from sklearn.model_selection import train_test_split
#dataset_url = '/content/small .csv'
#data = pd.read_csv(dataset_url)
#y = data.raw
#y_train, y_test = train_test_split(y,test_size=0.2)
train_dataloader = Data('/content/small .csv')
print("Train :",train_dataloader.__len__())
val_dataloader = Data('/content/large .csv')
print("Train :",val_dataloader.__len__())



Train : 830
Train : 830


In [ ]:
model = AutoEncoder()
criterion1 = nn.L1Loss()
criterion2 = nn.MSELoss()

def loss_fn(predicted,target):
    return criterion2(predicted,target)

@T.no_grad()
def validation(model, loader, loss_fn):
    vlosses = []
    v = tqdm(loader)
    model.eval()
    for i,(input_image,gt_image) in enumerate(v):
        input_image,gt_image = input_image.to(device), gt_image.to(device)
        y_pred = model(input_image)
        vloss = loss_fn(y_pred,gt_image)
        vlosses.append(vloss.item())
    return np.array(vlosses).mean()
    
model.load_state_dict(T.load("/content/model2.pth",map_location=T.device('cuda')))
optimizer = T.optim.Adam(model.parameters(), lr=0.001)

scheduler = T.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.1,patience=3,verbose=True)

In [ ]:
batch_size = 30
EPOCHES = 30
train_loader = DataLoader(train_dataloader,batch_size=batch_size,shuffle=False, num_workers=0, sampler=SubsetRandomSampler(list(range(train_dataloader.__len__()))),
                             drop_last=False)
val_loader = DataLoader(val_dataloader,batch_size=batch_size,shuffle=False,
                              num_workers=0,
                              sampler=SubsetRandomSampler(list(range(len(val_dataloader.dataset)))),
                             drop_last=False)

best_loss = None
raw_line0 = r'''Epoch[{}]    |    Lr:{}'''
raw_line1 = r'''TOTAL Train loss: {}  |  TOTAL Val loss: {}  |  Time:{:.1f} min '''

In [ ]:
 path_test1 = get_filepaths("/content/drive/MyDrive/Induction_training_files/test")
 dp = pd.DataFrame({"raw":path_test1})
 dp.to_csv("qiux .csv")

In [ ]:
for epoch in range(1, EPOCHES+1):
        losses = []
        start_time = time.time()
        t = tqdm(train_loader)
        model.train()
        for i,(input_image,gt_image) in enumerate(t):
            input_image,gt_image = input_image.to(device), gt_image.to(device)
            optimizer.zero_grad()
            y_pred = model(input_image)
            loss = loss_fn(y_pred,gt_image)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        vloss = validation(model, val_loader, loss_fn)
        print(raw_line0.format(epoch,optimizer.param_groups[0]["lr"]))
        print(raw_line1.format(np.array(losses).mean(),vloss,(time.time()-start_time)/60**1))

        if best_loss == None:
            best_loss = vloss
            T.save(model.state_dict(), 'model2.pth')
            print("saving model ..")
        if vloss < best_loss:
            best_loss = vloss
            T.save(model.state_dict(), 'model2.pth')
            print("saving model ..")
        scheduler.step(vloss)

  0%|          | 0/28 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100%|██████████| 28/28 [00:43<00:00,  1.57s/it]


Epoch[1]    |    Lr:0.001
TOTAL Train loss: 0.15714856263782298  |  TOTAL Val loss: 0.011600439569779806  |  Time:1.6 min 
saving model ..


100%|██████████| 28/28 [00:42<00:00,  1.51s/it]


Epoch[2]    |    Lr:0.001
TOTAL Train loss: 0.021505268955869333  |  TOTAL Val loss: 0.008784700683983309  |  Time:1.6 min 
saving model ..


100%|██████████| 28/28 [00:42<00:00,  1.51s/it]


...

Epoch[28]    |    Lr:1.0000000000000004e-08
TOTAL Train loss: 0.015287261256682021  |  TOTAL Val loss: 0.010507834177198154  |  Time:1.5 min 


100%|██████████| 28/28 [00:41<00:00,  1.48s/it]


Epoch[29]    |    Lr:1.0000000000000004e-08
TOTAL Train loss: 0.01538237972584154  |  TOTAL Val loss: 0.010854362942544478  |  Time:1.5 min 


100%|██████████| 28/28 [00:41<00:00,  1.49s/it]

Epoch[30]    |    Lr:1.0000000000000004e-08
TOTAL Train loss: 0.015324107969978027  |  TOTAL Val loss: 0.010781428876465984  |  Time:1.5 min 


In [ ]:
class Data2(Dataset):
    def __init__(self, dir_=None, input_size=(224,224), output_size=(112, 112),prob = 0.15):
        super().__init__()
        self.directory = dir_
        self.dataset = pd.read_csv(self.directory)
        self.input_size = input_size
        self.output_size = output_size
        self.prob = prob
        self.input_size_x = self.input_size[0]
        self.input_size_y = self.input_size[1]
        self.MODEL_SCALE = self.input_size[0]//self.output_size[0]
        self.preprocess = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __len__(self): return len(self.dataset)

    def __getitem_internal__(self, idx, preprocess=True):
        target = self.dataset.iloc[idx]
        noise_image = cv2.imread(target["raw"])
        dimensions = noise_image.shape
        noise_image = cv2.resize(noise_image,self.input_size)
        noise_image = transforms.ToTensor()(np.array(noise_image))
        basename1 = os.path.basename(target["raw"])
        return (noise_image,basename1,dimensions)

    def __getitem__(self, idx):
        return self.__getitem_internal__(idx, True)
    
    def raw(self, idx):
        return self.__getitem_internal__(idx, False)

In [ ]:
test_dataloader = Data2("/content/qiux .csv")
print("Test :",test_dataloader.__len__())

Test : 60


In [ ]:
import os
from skimage.metrics import structural_similarity as ssim
from skimage import io

def evaluate(p_img,gt_img):
    s = ssim(p_img, gt_img, multichannel=True)
    return s

In [ ]:
import cv2
import matplotlib.pyplot as plt
model.load_state_dict(T.load("/content/model2.pth",map_location=T.device('cuda')))
model.eval()
ssim_t = []
for i in range(60):
    raw_normalized ,basename1 , dimension = test_dataloader[i]
    
    y_pred = model(raw_normalized.unsqueeze(dim=0))
    raw_normalized = raw_normalized.permute(1,2,0).numpy()
    y_pred = y_pred.squeeze(dim=0).permute(1,2,0).detach().cpu().numpy()
    path1 = os.path.join('/content/drive/MyDrive/ai enhanced validation imagres', basename1)
    y_pred = cv2.resize(y_pred, (dimension[1],dimension[0]))
    y_pred = cv2.convertScaleAbs(y_pred,alpha=(255.0))
    cv2.imwrite(path1,y_pred)
    #plt.imshow(y_pred)
    #plt.savefig(path1)
    #plt.show()
    #fig, ax = plt.subplots(1, 2,figsize=(16,4))
    #ax[0].imshow(raw_normalized)        
    #ax[0].set_xticks([])
    #ax[0].set_yticks([])
    #ax[1].imshow(y_pred)        
    #ax[1].set_xticks([])
    #ax[1].set_yticks([])
    #plt.show()

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
